----TODO-----
Actualizar feronomonas, sobre todo fitness por pieza
Pruebas Fitness

In [21]:
import copy
from typing import List, Dict

In [22]:
class PiezaClase:
    def __init__(self, pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos
    
"""     def _initialize(pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos """

'     def _initialize(pieza, x_pos, y_pos):\n        self.pieza = pieza\n        self.x_pos = x_pos\n        self.y_pos = y_pos '

In [23]:
def explore_adyacent(temp, x, y): #hay que marcar la celda inicial antes de llamarla
    area = 0
    # Verificar si la celda adyacente está dentro de los límites del espacio
    if x + 1 < len(temp) and temp[x+1][y] == 0:
        temp[x+1][y] = 1
        area += explore_adyacent(temp, x+1, y)
    if x - 1 >= 0 and temp[x-1][y] == 0:
        temp[x-1][y] = 1
        area += explore_adyacent(temp, x-1, y)
    if y + 1 < len(temp[0]) and temp[x][y+1] == 0:
        temp[x][y+1] = 1
        area += explore_adyacent(temp, x, y+1)
    if y - 1 >= 0 and temp[x][y-1] == 0:
        temp[x][y-1] = 1
        area += explore_adyacent(temp, x, y-1)
    if temp[x][y]:
        area += 1
    return area


def huecos (mat):
  matrix = copy.deepcopy(mat)
  array = []
  for i in range(len(mat)):
    for j in range(len(mat[0])):
      if matrix[i][j] == 0:
        matrix[i][j] = 1
        array.append(explore_adyacent(matrix, i, j))
  return array

space =  [[0, 1, 0, 1],
          [0, 1, 0, 1],
          [0, 0, 1, 0],
          [0, 1, 1, 1]]
temp_area = copy.deepcopy(space)
array = huecos(temp_area)
print(space)
print (temp_area)
print (array)

[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[5, 2, 1]


In [24]:
def clamp_to_zero(number):
    if number < 0:
        return 0
    else:
        return number

In [25]:
#funcion para imprimir una matriz mas esteticamente
def imprimir_matriz(matriz):
    # Encuentra la longitud máxima para cada columna
    max_lengths = [max(len(str(row[i])) for row in matriz) for i in range(len(matriz[0]))]

    # Imprime la matriz con los números alineados correctamente
    for row in matriz:
        for i, item in enumerate(row):
            print(str(item).rjust(max_lengths[i]), end=" ")
        print()  # Nueva línea después de cada fila


FITNESS DEFINITION

In [26]:
def fitnessSinPieza(space):
    espacios = huecos(space)
    value = 100
    value = value - len(espacios) * 5 #penalizar numero de huecos
    for hueco in espacios:
      value = value - (1/hueco) * 3
    return clamp_to_zero(value)

In [27]:
import copy

def colocar_pieza(space, pieza, x, y):
    temp = copy.deepcopy(space)  # Deep copy the space to avoid modifying the original
    piece_height = len(pieza)
    piece_width = len(pieza[0])
    #print("\n".join([" ".join(map(str, row)) for row in temp]))

    # Verifica si la pieza puede colocarse en las coordenadas (x, y)
    if x + piece_height > len(space) or y + piece_width > len(space[0]):
        #print("La pieza queda fuera del espacio, fitness deberia ser 0")
        return False  # Las coordenadas están fuera del espacio
    
    for i in range(piece_height):
        for j in range(piece_width):
            if space[x+i][y+j] == 1 :
                #print("La pieza se superpondría con otra en esta posición")
                return False  # La pieza se superpondría con otra pieza ya colocada
            temp[x+i][y+j] = pieza[i][j]  # Coloca la pieza en el espacio temporal
    
    # Copia el espacio temporal modificado de vuelta al espacio original
    for i in range(len(space)):
        for j in range(len(space[0])):
            space[i][j] = temp[i][j]
    
    return True  # La pieza se colocó con éxito en las coordenadas (x, y)

In [28]:
def fitness(space, pieza, x, y):
    temp = copy.deepcopy(space)  # Deep copy the space to avoid modifying the original
    valid = colocar_pieza(temp, pieza.pieza, x, y)
    espacios = huecos(temp)
    if not valid:
        #print("La pieza no se puede colocar en la posición dada, por lo que el fitness es 0.")
        return 0
    else:
        value = 100
        value -= len(espacios) * 5  # Penalizar el número de huecos
        #print(f"Número de huecos: {len(espacios)}")
        for hueco in espacios:
            value -= (1 / hueco) * 3
            #print(f"Hueco: {hueco}, Penalización: {(1/hueco) * 3}")
            #print(f"fitness value: {value}")
        return clamp_to_zero(value)


In [29]:
def obtener_fitness_para_todas_las_posiciones(space, pieza):
    fitness_matriz = [[0 for _ in range(len(space[0]))] for _ in range(len(space))]

    print("Hemos inicializado bien la matriz del fitness")

    for i in range(len(space)):
        for j in range(len(space[0])):
            temp_space = [row[:] for row in space]  # Copia profunda de la matriz original
            colocada = colocar_pieza(temp_space, pieza.pieza, i, j)  # Modificación aquí
            if colocada:
                #colocada va a ser true si se puede colocar la pieza en i,j
                #print("Empezamos a rellenar la matriz del fitness")
                #print("La pieza podría colocarse en esta posición, por lo que el fitness debe ser > 0.")
                fitness_matriz[i][j] = round (fitness(space, pieza, i, j), 2)  # Almacena el fitness sin la pieza
                #funcion de fitness ejecutada correctamente

    print("Hemos rellenado correctamente la matriz del fitness")
    # Imprime la matriz de fitness
    print("Matriz de fitness:")
    imprimir_matriz(fitness_matriz)
    

    return fitness_matriz

In [30]:
#Función para crear la pieza, empezamos creando piezas de un máximo de area de 3x3, para empezar con una versión sencilla del problema, haremos que las piezas inicialmente solo puedan ser cuadrados de 1x1, 2x2 o 3x3.

import random

def generar_pieza():
    # Inicializamos una matriz vacía
    sizex = random.randint(1, 6)
    sizey = random.randint(1, 6)
    pieza = [[1 for column in range(sizex)] for row in range(sizey)]
    return pieza


def imprimir_pieza(pieza:PiezaClase):
    for row in pieza.pieza:
        print("  ".join(map(str, row)))
    print("coordenadas: x:",pieza.x_pos, "y:",pieza.y_pos)
    print()

def imprimir_espacio(space):
    for row in space:
        print("  ".join(map(str, row)))
    print()



# Ejemplo de uso
if __name__ == "__main__":
    pieza = PiezaClase(generar_pieza(),random.randint(1, 6),random.randint(1, 6))
    imprimir_pieza(pieza)

1
coordenadas: x: 2 y: 3



TEST DE FITNESS:

In [33]:
def test_fitness_function():
    # Genera un espacio pequeño para las pruebas
    print("Empezamos a generar el espacio.")
    x = 16
    y = 16
    space = [[0 for _ in range(x)] for _ in range(y)]

    # Genera un conjunto de piezas para las pruebas
    print("Procedemos a generar las piezas.")
    piezas = []
    for i in range(3):  # Limita el número de piezas para mantener las pruebas manejables
        # Genera una pieza individual con diferentes posiciones x e y en cada iteración
        piezaIndividual = PiezaClase(generar_pieza(), i, i)
        piezas.append(piezaIndividual)
    print("Piezas generadas correctamente.")    

    # Realiza las pruebas de la función de fitness para cada pieza
    print("Empezamos las pruebas.")
    for pieza in piezas:
        x_pos = pieza.x_pos
        y_pos = pieza.y_pos

        # Imprime la matriz de espacio antes de colocar la pieza
        print("Matriz de espacio antes de colocar la pieza:")
        imprimir_espacio(space)

        # Imprime la pieza antes de intentar colocarla
        print("Pieza a colocar:")
        imprimir_espacio(pieza.pieza)

        # Llama a la función para obtener el fitness para todas las posiciones de la matriz
        print("Llamamos a la funcion para obtener la matriz de fitness.")
        fitness_matriz = obtener_fitness_para_todas_las_posiciones(space, pieza)

        print("La matriz de fitness se ha rellenado correctamente \n")

        # Coloca la pieza en el espacio
        print("Comprobamos si podemos colocar la pieza:")
        colocada = False
        for i in range(len(space)):  # Recorre todas las posiciones del espacio
            for j in range(len(space[0])):
                colocada = colocar_pieza(space, pieza.pieza, i, j)
                if colocada:  # Si la pieza fue colocada, sale del bucle
                    print("Pieza colocada exitosamente.")
                    break
            if colocada:
                break  # Sale del bucle externo si la pieza fue colocada
        if not colocada:
            print("La pieza no se puede colocar en ninguna posición dada.")

        # Imprime la matriz de espacio después de colocar la pieza
        print("Matriz de espacio después de colocar la pieza:")
        imprimir_espacio(space)


test_fitness_function()


Empezamos a generar el espacio.
Procedemos a generar las piezas.
Piezas generadas correctamente.
Empezamos las pruebas.
Matriz de espacio antes de colocar la pieza:
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

Pieza a colocar:
1  1  1  1  1  1
1  1  1  1  1  1

Llamamos a la funcion para obt

In [32]:

x = 16
y = 16
space = [[0 for column in range(x)] for row in range(y)]

x_pos = 0
y_pos = 0
#print(x_pos,y_pos)
#pieza = generar_pieza()
pieza2 = PiezaClase(generar_pieza(),0,0)
imprimir_pieza(pieza2)
print(fitness(space, pieza2.pieza, pieza2.x_pos, pieza2.y_pos))


print("inicial: ")
imprimir_espacio(space)
temp_area = copy.deepcopy(space)
array = huecos(temp_area)

print (temp_area)
print (array)

1  1
1  1
coordenadas: x: 0 y: 0



AttributeError: 'list' object has no attribute 'pieza'

PRIMERA IMPLEMENTACION DE ACO

In [ ]:
#Genero el espacio donde colocamos las piezas
x = 16
y = 16
space = [[0 for column in range(x)] for row in range(y)]

#Genero un array de piezas
piezas = []
for i in range(10):
    piezaIndividual = PiezaClase(generar_pieza(),random.randint(0, 15),random.randint(0, 15))
    piezas.append(piezaIndividual)
    
    #imprimir_pieza(piezas[i])

In [ ]:
def heuristico(piezaHeu:PiezaClase): #se puede hacer un heuristico que varíe segun avanza.
    size = len(piezaHeu.pieza)*len(piezaHeu.pieza[0])
    
    return size / (x*x)

In [ ]:
def calculoProbabilidades(pheromonesCopy, piezas):
    alpha = 0.5
    beta = 0.5
    dividendo = []
    suma = 0
    for i in range(0, len(piezas)):
        dividendo.append(pow(pheromonesCopy[i],alpha) * pow(heuristico(piezas[i]),beta))
        suma += dividendo[i]
    for j in dividendo:
        j = j/suma
    print("Dividendo",dividendo)
    return dividendo

In [ ]:
#El array sol contiene n arrays, donde n es len de piezas, en los cuales el primer valor es
# si se pone o no se pone le pieza y los otros dos la posicion
# En caso de que se ponga se intenta colocar por ahora de manera aleatoria 50 veces, si no se consigue se pone como False
# esto despues se hara con feromonas y tal pero por ahora random
def generar_sol(space, pheromones):
    sol = [[0 for column in range(0,3)] for row in range(len(piezas))]
    probabilidades=calculoProbabilidades(copy.deepcopy(pheromones), piezas)
    for i in range(len(piezas)):
        if(random.random() < probabilidades[i]):
            sol[i][0] = True
        else:
            sol[i][0] = False
        if (sol[i][0] == True):
            for j in range (0,50):
                posTempX = random.randint(0, 15)
                posTempY = random.randint(0, 15)
                if(fitness(space,piezas[i].pieza,posTempX, posTempY) != 0):
                    sol[i][1] = posTempX
                    sol[i][2] = posTempY
                    break
                elif(j == 10):
                    sol[i][0] = False
                    sol[i][1] = -1
                    sol[i][2] = -1
        else:
            sol[i][1] = -1
            sol[i][2] = -1
    return sol

# print(generar_sol(space))
# imprimir_espacio(space)
# print(fitnessSinPieza(space))

In [ ]:
#Genero el ACO
def clamp(value):
    return min(value, 1)
pheromones = [clamp(random.random()+0.1, 1) for i in range(0,len(piezas))]
best_solution = []
numberAnts = 10
for i in range(0,1):
    caminos = []
    for j in range(numberAnts):
        solution = generar_sol(copy.deepcopy(space), pheromones)
        print(fitnessSinPieza(solution), solution) #TO DO, imprimir space con esta sol para leerlo mejor
        caminos.append(solution)
        fit = fitnessSinPieza(solution)
        if(fit > fitnessSinPieza(best_solution)):
            best_solution = solution
    

imprimir_espacio(space)

NameError: name 'piezas' is not defined

In [ ]:
#ACO template

## Constants ##
Board_side = 100
Num_Pieces = 20
Alpha  = 0.5
Beta   = 0.5

## Utilities ##
def clamp(value):
    if value > 0:
        return min(value, 1)
    else: 
        return max (value, 0)

## Classes ##
class Piece:
    def __init__(self, x_dim, y_dim):
        self.x_dim = x_dim
        self.y_dim = y_dim

    def generate_random_pieces(cardinality, max_size):
        pieces = []
        for i in range(cardinality):
            pieces.append(Piece(random.randint(0, max_size),random.randint(0, max_size)))
        return pieces

class Choice:
    def __init__(self, piece, x_pos, y_pos):
        self.piece = piece
        self.x_pos = x_pos
        self.y_pos = y_pos

class Solution:
    def __init__(self, x_dim, y_dim):
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.board = [[False for column in range(x_dim)] for row in range(y_dim)]
        self.used_pieces = [False for row in range(Num_Pieces)]

    def does_Choice_fit(self, choice):
        if choice.piece.x_dim + choice.x_pos > self.x_dim:
            return False
        if choice.piece.y_dim + choice.y_pos > self.y_dim:
            return False
        for i in range(choice.piece.x_dim):
            for j in range(choice.piece.y_dim):
                if self.board[choice.x_pos + i][choice.y_pos + j]:
                    return False
        return True

    def place_Choice(self, choice):
        for i in range(choice.piece.x_dim):
            for j in range(choice.piece.y_dim):
                self.board[choice.x_pos + i][choice.y_pos + j] = True
        self.used_pieces[Pieces.index(choice.piece)] = True

    def does_use_piece(self, piece):
        return self.used_pieces[Pieces.index(piece)]

## Global values ##
Pieces = Piece.generate_random_pieces()


## Functions ##
def initialize_pheromones():
    Pheromones = ??
    return Pheromones

def update_pheromones(Pheromones, Solution_list):
    New_Pheromones = initialize_pheromones()
    ??
    return New_Pheromones

def evaluate_fitness(solution):
    fitness = 0
    ??
    return fitness

def get_candidates(solution):
    Candidates = []
    for candidate in Pieces:
        if not solution.does_use_piece(candidate):
            ?? #select a position
            choice = Choice(candidate, x, y)
            if solution.does_Choice_fit(choice):
                Candidates.append(choice)               
    return Candidates

def heuristic(choice): #good place for experimentation
    size = choice.piece.x_dim * choice.piece.y_dim
    return size / (Board_side * Board_side)

def calculate_probabilities(Candidates, Pheromones):
    dividend = []
    sum = 0
    for i in range(0, len(Candidates)):
        dividend.append(pow(Pheromones[i],Alpha) * pow(heuristic(Candidates[i]),Beta))
        sum += dividend[i]
    for j in dividend:
        j = j/sum
    return dividend

def random_choice(Candidates, Probabilities):
    for i in range(len(Candidates)):
        if random.random() < Probabilities[i]: #good option for experimenting an using other choosing algos
            return Candidates[i]

def construct_solution(Pheromones):
    Solution = []
    Candidates = get_candidates(Solution) #candidates are always valid
    while (not Candidates.empty()):
        Probabilities = calculate_probabilities(Candidates, Pheromones)
        choice = random_choice(Candidates, Probabilities)
        Solution.place_Choice(choice)
        Candidates = get_candidates(Solution)

    return Solution

def ACO(n_ants):
    Pheromones = initialize_pheromones()
    best_fitness = 0
    best_solution = []
    while (not has_finished()):
        Solution_list = []
        for i in range(n_ants):
            solution = construct_solution(Pheromones)
            Solution_list.append(solution)
            fintess = evaluate_fitness(solution)
            if (fintess > best_fitness):
                best_solution = solution
                best_fitness = fitness

        Pheromones = update_pheromones(Pheromones, Solution_list)

    return best_solution





#
#
#
#
#
#
#
#
#
#
#
#
#

SyntaxError: invalid syntax (55070495.py, line 53)